# Cell Segmentation Pipeline – Notebook Overview

This notebook demonstrates how to **batch process Cell Painting images** using a combination of deep learning (Cellpose) and classical image processing (thresholding).

We will:

- Use Cellpose (`cyto3`, `nuclei`) for cell and nuclei segmentation
- Use Otsu thresholding for mitochondria segmentation
- Extract shape and intensity features using `regionprops_table`
- Handle different file naming conventions
- Save segmentation masks and feature tables for each image


## Setup for Google Colab

If you're running this notebook in Google Colab, run the cell below to install the required Python packages:

In [ ]:
!pip install -q imageio[ffmpeg] scikit-image matplotlib cellpose stardist csbdeep

In [ ]:
import pandas as pd
import numpy as np
import re
import os

from pathlib import Path
from skimage.io import imread
from skimage.exposure import rescale_intensity
from skimage.morphology import disk, remove_small_objects
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops_table
from skimage.filters.rank import median
from cellpose import models
from imageio import imwrite
from tqdm import tqdm

### Initialize Models and Paths

- Initialize **Cellpose** models:
  - `'nuclei'` for DNA segmentation
  - `'cyto3'` for cytoplasm/cell segmentation
- Defines the base folder (`BASE_PATH`) where the dataset is located
- Lists all experimental classes (treatment groups)


In [ ]:
# Initialize Cellpose models
model_nuclei = models.Cellpose(model_type='nuclei', gpu=False)
model_cyto3 = models.Cellpose(model_type='cyto3', gpu=False)

# Base path
BASE_PATH = Path("") # TODO: add path to local folder
CLASSES = ['Brefeldin-A-like', 'Etoposide', 'Nocodazole', 'Rapamycin-like', 'Staurosporine-like']

### Utility Functions

- `load_image`, `load_and_preprocess_mito_image`: image loading and enhancement
- `ensure_segmentation_folder`: creates a subfolder for storing segmentation masks (with Cellpose parameters in the name)
- `save_mask`: saves segmentation mask as 16-bit image
- `get_props`: extracts all available region properties from masks (including intensity if image is passed)
- `parse_standard_class`: parses standard image naming for 4 classes
- `parse_staurosporine_class`: handles Staurosporine-like naming convention
- `segment_nuclei`: runs Cellpose on DNA channel
- `segment_cells`: runs Cellpose on AGP+ER+DNA merged channels
- `segment_mito`: segments mitochondria using Otsu thresholding


In [ ]:
# Function to load image
def load_image(path):
    return rescale_intensity(imread(str(path)), out_range=(0, 255)).astype(np.uint8)

def load_and_preprocess_mito_image(path):
    img = rescale_intensity(imread(str(path)), out_range=(0, 255)).astype(np.uint8)
    img = median(img, disk(2))
    #img = (equalize_adapthist(img, clip_limit=0.01) * 255).astype(np.uint8)
    img = rescale_intensity(img, out_range=(0, 255)).astype(np.uint8)

    return img

def ensure_segmentation_folder(image_path, prefix, diameter, flow_threshold, cellprob_threshold):
    seg_name = f"segmentation_{prefix}_d{diameter}_f{flow_threshold}_cp{cellprob_threshold}"
    seg_dir = image_path.parent / seg_name
    seg_dir.mkdir(exist_ok=True)
    return seg_dir / image_path.name

# Save mask
def save_mask(mask, out_path):
    imwrite(out_path, (mask.astype(np.uint16)))

# Extract regionprops
from skimage.measure import label, regionprops_table
import pandas as pd

def get_props(mask, intensity_img, filename, cls):
    lbl = label(mask)

    # Full set of regionprops properties
    properties = [
        # Labeling
        "label",

        # Shape / Morphology
        "area",
        "bbox",
        "bbox_area",
        "convex_area",
        "eccentricity",
        "equivalent_diameter",
        "extent",
        "filled_area",
        "major_axis_length",
        "minor_axis_length",
        "orientation",
        "perimeter",
        "solidity",

        # Centroids & Coordinates
        "centroid",
        "weighted_centroid",

        # Intensity-based (only if intensity image is provided)
        "mean_intensity",
        "max_intensity",
        "min_intensity"
    ]

    if intensity_img is not None:
        props = regionprops_table(lbl, intensity_image=intensity_img, properties=properties)
    else:
        # Remove intensity-based props if no intensity image is passed
        props = regionprops_table(
            lbl,
            properties=[p for p in properties if not p.endswith("intensity")]
        )

    df = pd.DataFrame(props)
    df["filename"] = filename.name
    df["class"] = cls
    return df

# Parse files for 4 classes (standard naming)
def parse_standard_class(folder):
    files_by_id = {}
    subdirs = ["DNA", "Mito", "ER", "AGP"]
    pattern = re.compile(r'^(?P<id>CP\d-SC\d-\d+_[A-Z]\d{2}_T\d{4}F\d{3})[^C]*C(?P<ch>\d{2})')

    for subdir in subdirs:
        files = list((folder / "Orig" / subdir).glob("*.tif"))
        for f in files:
            m = pattern.match(f.name)
            if m:
                base_id = m.group("id")
                ch = m.group("ch")
                if base_id not in files_by_id:
                    files_by_id[base_id] = {}
                files_by_id[base_id][subdir] = f
    return files_by_id

# Parse files for Staurosporine-like (custom naming)
def parse_staurosporine_class(folder):
    files_by_id = {}
    channel_map = {
        "1": "Mito",
        "2": "AGP",
        "3": "RNA",
        "4": "ER",
        "5": "DNA"
    }

    all_files = list((folder / "Orig").rglob("*.tif")) + list((folder / "Orig").rglob("*.tiff"))

    pattern = re.compile(r'(?P<id>r\d{2}c\d{2}f\d{2}p\d{2})-ch(?P<ch>\d)')

    for f in all_files:
        match = pattern.search(f.name)
        if match:
            base_id = match.group("id")
            ch = match.group("ch")
            label = channel_map.get(ch)
            if label:
                if base_id not in files_by_id:
                    files_by_id[base_id] = {}
                files_by_id[base_id][label] = f
    return files_by_id

def segment_nuclei(dna_img, diameter=30, flow_threshold=0.4, cellprob_threshold=0.0, channels=[0, 0]):
    mask, _, _, _ = model_nuclei.eval(
        dna_img,
        diameter=diameter,
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        channels=channels
    )
    return mask

def segment_cells(agp_img, er_img, dna_img, diameter=45, flow_threshold=0.4, cellprob_threshold=0.0, channels=[0, 1]):
    merged = np.stack([agp_img, er_img, dna_img], axis=-1)
    mask, _, _, _ = model_cyto3.eval(
        merged,
        diameter=diameter,
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold,
        channels=channels
    )
    return mask

def segment_mito(mito_img):
    if mito_img.max() == mito_img.min():
        # Avoid Otsu error on flat image
        return np.zeros_like(mito_img, dtype=np.uint8)
    
    thresh = threshold_otsu(mito_img)
    mask = (mito_img > thresh).astype(np.uint8)
    return mask

### Batch Processing Loop:

1. Matches image sets by ID
2. Loads and preprocesses channels (DNA, AGP, ER, Mito)
3. Applies segmentation for:
   - **Nuclei** (Cellpose `nuclei`)
   - **Cells** (Cellpose `cyto3`)
   - **Mito** (Otsu thresholding)
4. Saves segmentation masks to output folders with parameter info
5. Extracts region properties (shape + intensity) using `get_props`
6. Appends results to class-specific DataFrames


In [ ]:
# Process all classes
all_nuclei, all_cells, all_mito = [], [], []

for cls in CLASSES:
    print(cls)
    folder = BASE_PATH / cls 
    if cls == "Staurosporine-like":
        matched = parse_staurosporine_class(folder)
    else:
        matched = parse_standard_class(folder)

    #for img_id, channels in matched.items():
    for img_id, channels in tqdm(matched.items(), desc=f"Processing {cls}"):
        #class_name = cls
        dna_path = channels.get("DNA")
        agp_path = channels.get("AGP")
        er_path  = channels.get("ER")
        mito_path = channels.get("Mito")
    
        # Skip if critical images are missing
        if not (dna_path and agp_path and er_path and mito_path):
            continue
    
        # --- Load images ---
        dna = load_and_preprocess_mito_image(dna_path)
        agp = load_and_preprocess_mito_image(agp_path)
        er  = load_and_preprocess_mito_image(er_path)
        mito = load_and_preprocess_mito_image(mito_path)
    
        # --- Segment nuclei ---
        mask_nuclei = segment_nuclei(dna, diameter=20, flow_threshold=0.4, cellprob_threshold=-2)
        out_nuclei = ensure_segmentation_folder(dna_path, "nuclei", 20, 0.4, -2.0)
        save_mask(mask_nuclei, out_nuclei)
        all_nuclei.append(get_props(mask_nuclei, dna, dna_path, cls))
    
        # --- Segment cells ---
        cells_merged = np.stack([agp, er], axis=-1)
        mask_cells = segment_cells(agp, er, dna, diameter=50, flow_threshold=0.5, cellprob_threshold=-4)
        out_cells = ensure_segmentation_folder(agp_path, "cells", 50, 0.5, -4.0)
        save_mask(mask_cells, out_cells)
        all_cells.append(get_props(mask_cells, cells_merged, agp_path, cls))
    
        # --- Segment mito (Otsu) ---
        mask_mito = segment_mito(mito)
        out_mito = ensure_segmentation_folder(mito_path, "mito", 0, 0, 0)
        save_mask(mask_mito, out_mito)
        all_mito.append(get_props(mask_mito, mito, mito_path, cls))

# Concatenate all results
df_nuclei = pd.concat(all_nuclei, ignore_index=True)
df_cells = pd.concat(all_cells, ignore_index=True)
df_mito = pd.concat(all_mito, ignore_index=True)

In [ ]:
output_dir = BASE_PATH / "processed"
output_dir.mkdir(exist_ok=True)

df_nuclei.to_csv(output_dir / "features_nuclei.csv", index=False)
df_cells.to_csv(output_dir / "features_cells.csv", index=False)
df_mito.to_csv(output_dir / "features_mito.csv", index=False)

print("Feature tables saved to:", output_dir)